# 🔹 UFC Fight Predictor Model Training

<div style="text-align: center;">
  🔹 <img src="../img/ufc_logo.png" width="50" /> 🔹
</div>

#  Import Libraries and Setup Environment

In [1]:
# Import necessary libraries
import os
import sys
import pandas as pd

pd.set_option('display.max_colwidth', 200) 

# Get the current working directory
current_dir = os.getcwd()

# Navigate to the project root
project_root = os.path.abspath(os.path.join(current_dir, '..'))

# Import from /src
sys.path.append(os.path.join(project_root))
from src.metrics import *
from src.model_factory import *
from src.io_model import *
from src.helpers import *
from src.config import *

<div style="text-align: center;">
  🔹 <img src="../img/ufc_logo.png" width="50" /> 🔹
</div>

#  Load Data

In [2]:
# Load UFCData
try:
    ufc_data = load_data(name='ufc_data')
    ufc_data_no_odds = load_data(name='ufc_data_no_odds')
    logger.info("✅ UFCData objects loaded successfully.")
except Exception as e:
    logger.error(f"❌ Error loading training data: {e}")

INFO:src.io_model:📦 UFCData object loaded from: /home/mlioi/ufc-predictor/data/processed/ufc_data.pkl
INFO:src.io_model:📦 UFCData object loaded from: /home/mlioi/ufc-predictor/data/processed/ufc_data_no_odds.pkl
INFO:src.helpers:✅ UFCData objects loaded successfully.


In [3]:
ufc_data

📊 UFC Dataset Summary
----------------------------------------
🧪 Total samples      : 6001
🧪 Train/Test split  : 4800 / 1201
🧪 Total features     : 20

🔢 Numerical features : 17
🔠 Categorical features: 3
    - Binary          : 2
    - Multiclass      : 1

🏷 Label distribution (raw):
   - Class 0: 3484 (58.1%)
   - Class 1: 2517 (41.9%)

✅ No missing values detected

📈 Feature summary statistics (train set):
                        mean      std       min       max
LoseStreakDif          0.059    1.012    -6.000     6.000
WinStreakDif          -0.162    1.921   -18.000    10.000
TotalTitleBoutDif     -0.313    1.709   -16.000    15.000
KODif                 -0.536    2.181   -21.000    14.000
SubDif                -0.334    1.877   -15.000    10.000
HeightDif             -0.030    6.286   -33.020    30.480
ReachDif              -0.260    8.266   -33.020    30.480
AgeDif                 0.196    5.213   -17.000    17.000
SigStrDif             -2.540   19.869  -118.000   128.222
AvgSubAt

In [4]:
ufc_data_no_odds

📊 UFC Dataset Summary
----------------------------------------
🧪 Total samples      : 6001
🧪 Train/Test split  : 4800 / 1201
🧪 Total features     : 19

🔢 Numerical features : 16
🔠 Categorical features: 3
    - Binary          : 2
    - Multiclass      : 1

🏷 Label distribution (raw):
   - Class 0: 3484 (58.1%)
   - Class 1: 2517 (41.9%)

✅ No missing values detected

📈 Feature summary statistics (train set):
                      mean     std      min      max
LoseStreakDif        0.059   1.012   -6.000    6.000
WinStreakDif        -0.162   1.921  -18.000   10.000
TotalTitleBoutDif   -0.313   1.709  -16.000   15.000
KODif               -0.536   2.181  -21.000   14.000
SubDif              -0.334   1.877  -15.000   10.000
HeightDif           -0.030   6.286  -33.020   30.480
ReachDif            -0.260   8.266  -33.020   30.480
AgeDif               0.196   5.213  -17.000   17.000
SigStrDif           -2.540  19.869 -118.000  128.222
AvgSubAttDif        -0.076   0.880   -8.400    7.800
AvgTD

<div style="text-align: center;">
  🔹 <img src="../img/ufc_logo.png" width="50" /> 🔹
</div>

# 🔧 Hyperparameters Tuning 🔧

In [5]:
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

hyperparameter_tuning = True

if hyperparameter_tuning:
    logger.info("💡 Hyperparameters Tuning is Activated")

    model_params = {
    "Support Vector Machine": (
        SVC(probability=True),
        {'C': [0.01, 0.1, 1, 1.5, 2], 'kernel': ['linear', 'rbf'], 'gamma': ['auto']}
    ),
    "Random Forest": (
        RandomForestClassifier(),
        {'n_estimators': [10, 50, 100, 1000], 'max_depth': [3, 5, 10, 100]}
    ),
    "Logistic Regression": (
        LogisticRegression(),
        {'C': [0.001, 0.01, 0.05, 1], 'solver': ['liblinear'], 'penalty': ['l1','l2']}
    ),
    "K-Nearest Neighbors": (
        KNeighborsClassifier(),
        {'n_neighbors': [3, 6, 9, 12, 15], 'weights': ['uniform', 'distance'], 'metric': ['euclidean', 'manhattan']}
    ),
    "AdaBoost": (
        AdaBoostClassifier(),
        {'n_estimators': [10, 50, 100], 'learning_rate': [0.01, 0.1, 1.0]}
    ),
    "Naive Bayes": (
        GaussianNB(),
        {'var_smoothing': [1e-8, 1e-7, 1e-6, 1e-5]}
    ),
    "Gradient Boosting": (
        GradientBoostingClassifier(),
        {'n_estimators': [50, 100], 'learning_rate': [0.01, 0.1, 1.0], 'max_depth': [3, 5, 7]}
    ),
    "Extra Trees": (
        ExtraTreesClassifier(),
        {'n_estimators': [50, 100], 'max_depth': [None, 10, 20]}
    ),
    "Quadratic Discriminant Analysis": (
        QuadraticDiscriminantAnalysis(),
        {'reg_param': [0.1, 1]}
    ),
    "Neural Network": (
        MLPClassifier(max_iter=200, random_state=42),
        {
        # Architecture: number and size of hidden layers
        'hidden_layer_sizes': [
            (200,), (50, 50),
        ],
        # Activation function for hidden layers
        'activation': ['relu', 'logistic'],
        # Optimizer for gradient descent
        'solver': ['adam', 'sgd'],
        # L2 regularization strength (higher alpha reduces overfitting but can cause underfitting)
        'alpha': [0.001, 0.01],
        # Learning rate schedule
        'learning_rate': ['adaptive'],
        # Initial learning rate (for adam and sgd solvers)
        'learning_rate_init': [0.01],
        # Early stopping based on validation performance to prevent overfitting
        'early_stopping': [True],
        # Mini-batch size for training with adam/sgd
        'batch_size': [32, 64],
        # Momentum for SGD (controls contribution of previous updates)
        'momentum': [0.8],
        # Proportion of training set used as validation for early stopping
        'validation_fraction': [0.2]
        }
    ),
    "XGBoost": (
            XGBClassifier(eval_metric='logloss'),
            {
                'n_estimators': [50, 150],
                'learning_rate': [0.01, 0.1],
                'max_depth': [3, 5, 7],
                'subsample': [0.8, 1.0],
                'colsample_bytree': [0.8, 1.0]
            }
        )
}
    display_model_params_table(model_params)
else:
    logger.info("💡 Hyperparameters Tuning is Deactivated")
    model_params = None # Default Settings

INFO:src.helpers:💡 Hyperparameters Tuning is Activated


,Model,Hyperparameters
0,Support Vector Machine,"C: [0.01, 0.1, 1, 1.5, 2]; kernel: ['linear', 'rbf']; gamma: ['auto']"
1,Random Forest,"n_estimators: [10, 50, 100, 1000]; max_depth: [3, 5, 10, 100]"
2,Logistic Regression,"C: [0.001, 0.01, 0.05, 1]; solver: ['liblinear']; penalty: ['l1', 'l2']"
3,K-Nearest Neighbors,"n_neighbors: [3, 6, 9, 12, 15]; weights: ['uniform', 'distance']; metric: ['euclidean', 'manhattan']"
4,AdaBoost,"n_estimators: [10, 50, 100]; learning_rate: [0.01, 0.1, 1.0]"
5,Naive Bayes,"var_smoothing: [1e-08, 1e-07, 1e-06, 1e-05]"
6,Gradient Boosting,"n_estimators: [50, 100]; learning_rate: [0.01, 0.1, 1.0]; max_depth: [3, 5, 7]"
7,Extra Trees,"n_estimators: [50, 100]; max_depth: [None, 10, 20]"
8,Quadratic Discriminant Analysis,"reg_param: [0.1, 1]"
9,Neural Network,"hidden_layer_sizes: [(200,), (50, 50)]; activation: ['relu', 'logistic']; solver: ['adam', 'sgd']; alpha: [0.001, 0.01]; learning_rate: ['adaptive']; learning_rate_init: [0.01]; early_stopping: [T..."


# Notes
- For SVM, kernel='linear' has better performance.
- For Logistic Regression, performs well for lower values of c
- Neural network with shape (50x50 works great)
- SVM has a more rich feature importance results with odds
- SVM Linear without odds turns out to be a dummy model
- SVM RBF is better for no odds model

<div style="text-align: center;">
  🔹 <img src="../img/ufc_logo.png" width="50" /> 🔹
</div>

# 🔹 UFC Machine Learning Training (with Odds)

##  KNN Model Training

### 🚀 KNN Training 

In [6]:
model_name = 'K-Nearest Neighbors'
start = time.time()
model = model_factory(model_name, ufc_data, model_params, scoring = 'accuracy')
end = time.time()
duration = end - start

INFO:src.model_factory:[K-Nearest Neighbors] UFC GridSearchCV Training (Accuracy)
INFO:root:[K-Nearest Neighbors] 🤖 Training...


Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV 1/5] END metric=euclidean, n_neighbors=3, weights=uniform;, score=0.570 total time=   0.1s
[CV 2/5] END metric=euclidean, n_neighbors=3, weights=uniform;, score=0.591 total time=   0.1s
[CV 3/5] END metric=euclidean, n_neighbors=3, weights=uniform;, score=0.621 total time=   0.1s
[CV 4/5] END metric=euclidean, n_neighbors=3, weights=uniform;, score=0.597 total time=   0.1s
[CV 5/5] END metric=euclidean, n_neighbors=3, weights=uniform;, score=0.599 total time=   0.1s
[CV 1/5] END metric=euclidean, n_neighbors=3, weights=distance;, score=0.570 total time=   0.0s
[CV 2/5] END metric=euclidean, n_neighbors=3, weights=distance;, score=0.593 total time=   0.0s
[CV 3/5] END metric=euclidean, n_neighbors=3, weights=distance;, score=0.621 total time=   0.0s
[CV 4/5] END metric=euclidean, n_neighbors=3, weights=distance;, score=0.597 total time=   0.0s
[CV 5/5] END metric=euclidean, n_neighbors=3, weights=distance;, score=0.598 to

INFO:root:[K-Nearest Neighbors] 🔍 Best Accuracy: 0.6250
[K-Nearest Neighbors] 🔍 Best Params: {'metric': 'euclidean', 'n_neighbors': 15, 'weights': 'distance'}


### 🔍 KNN Metrics

In [7]:
metrics = evaluate_metrics(model, ufc_data)

### 📝 Log Training Results

In [8]:
log_training_result(model_name, model.best_params_, metrics, duration)

INFO:src.helpers:✅ Training logged to ../data/results/training_log.csv


### 💾 Save Model 

In [9]:
save_model(model, name=file_model_name[model_name])

INFO:src.io_model:✅ Model 'K-Nearest Neighbors' saved to: /home/mlioi/ufc-predictor/models/knn_best.pkl


## Support Vector Machine Model 

### 🚀 Support Vector Machine Training 

In [10]:
model_name = 'Support Vector Machine'
start = time.time()
model = model_factory(model_name, ufc_data, model_params)
end = time.time()
duration = end - start

INFO:src.model_factory:[Support Vector Machine] UFC GridSearchCV Training (Accuracy)
INFO:root:[Support Vector Machine] 🤖 Training...


Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV 1/5] END .C=0.01, gamma=auto, kernel=linear;, score=0.652 total time=   2.6s
[CV 2/5] END .C=0.01, gamma=auto, kernel=linear;, score=0.643 total time=   2.7s
[CV 3/5] END .C=0.01, gamma=auto, kernel=linear;, score=0.692 total time=   2.7s
[CV 4/5] END .C=0.01, gamma=auto, kernel=linear;, score=0.665 total time=   2.6s
[CV 5/5] END .C=0.01, gamma=auto, kernel=linear;, score=0.659 total time=   2.7s
[CV 1/5] END ..C=0.1, gamma=auto, kernel=linear;, score=0.654 total time=   3.0s
[CV 2/5] END ..C=0.1, gamma=auto, kernel=linear;, score=0.637 total time=   3.0s
[CV 3/5] END ..C=0.1, gamma=auto, kernel=linear;, score=0.688 total time=   3.1s
[CV 4/5] END ..C=0.1, gamma=auto, kernel=linear;, score=0.661 total time=   3.0s
[CV 5/5] END ..C=0.1, gamma=auto, kernel=linear;, score=0.659 total time=   3.0s
[CV 1/5] END ....C=1, gamma=auto, kernel=linear;, score=0.656 total time=   5.3s
[CV 2/5] END ....C=1, gamma=auto, kernel=linear;,

INFO:root:[Support Vector Machine] 🔍 Best Accuracy: 0.6621
[Support Vector Machine] 🔍 Best Params: {'C': 0.01, 'gamma': 'auto', 'kernel': 'linear'}


### 🔍 Support Vector Machine Metrics

In [11]:
metrics = evaluate_metrics(model, ufc_data)

### 📝 Log Training Results

In [12]:
log_training_result(model_name, model.best_params_, metrics, duration)

INFO:src.helpers:✅ Training logged to ../data/results/training_log.csv


### 💾 Save Model 

In [13]:
save_model(model, name=file_model_name[model_name])

INFO:src.io_model:✅ Model 'Support Vector Machine' saved to: /home/mlioi/ufc-predictor/models/svm_best.pkl


## Logistic Regression Model

### 🚀 Logistic Regression Training

In [14]:
model_name = 'Logistic Regression'
start = time.time()
model = model_factory(model_name, ufc_data, model_params)
end = time.time()
duration = end - start

INFO:src.model_factory:[Logistic Regression] UFC GridSearchCV Training (Accuracy)
INFO:root:[Logistic Regression] 🤖 Training...


Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV 1/5] END C=0.001, penalty=l1, solver=liblinear;, score=0.581 total time=   0.0s
[CV 2/5] END C=0.001, penalty=l1, solver=liblinear;, score=0.581 total time=   0.0s
[CV 3/5] END C=0.001, penalty=l1, solver=liblinear;, score=0.580 total time=   0.0s
[CV 4/5] END C=0.001, penalty=l1, solver=liblinear;, score=0.580 total time=   0.0s
[CV 5/5] END C=0.001, penalty=l1, solver=liblinear;, score=0.580 total time=   0.0s
[CV 1/5] END C=0.001, penalty=l2, solver=liblinear;, score=0.656 total time=   0.0s
[CV 2/5] END C=0.001, penalty=l2, solver=liblinear;, score=0.636 total time=   0.0s
[CV 3/5] END C=0.001, penalty=l2, solver=liblinear;, score=0.682 total time=   0.0s
[CV 4/5] END C=0.001, penalty=l2, solver=liblinear;, score=0.657 total time=   0.0s
[CV 5/5] END C=0.001, penalty=l2, solver=liblinear;, score=0.669 total time=   0.0s
[CV 1/5] END C=0.01, penalty=l1, solver=liblinear;, score=0.657 total time=   0.0s
[CV 2/5] END C=0.

INFO:root:[Logistic Regression] 🔍 Best Accuracy: 0.6638
[Logistic Regression] 🔍 Best Params: {'C': 0.01, 'penalty': 'l1', 'solver': 'liblinear'}


### 🔍 Logistic Regression Metrics

In [15]:
metrics = evaluate_metrics(model, ufc_data)

### 📝 Log Training Results

In [16]:
log_training_result(model_name, model.best_params_, metrics, duration)

INFO:src.helpers:✅ Training logged to ../data/results/training_log.csv


### 💾 Save Model 

In [17]:
save_model(model, name=file_model_name[model_name])

INFO:src.io_model:✅ Model 'Logistic Regression' saved to: /home/mlioi/ufc-predictor/models/lr_best.pkl


## Random Forest Model

### 🚀 Random Forest Training

In [18]:
model_name = 'Random Forest'
start = time.time()
model = model_factory(model_name, ufc_data, model_params)
end = time.time()
duration = end - start

INFO:src.model_factory:[Random Forest] UFC GridSearchCV Training (Accuracy)
INFO:root:[Random Forest] 🤖 Training...


Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV 1/5] END ......max_depth=3, n_estimators=10;, score=0.642 total time=   0.0s
[CV 2/5] END ......max_depth=3, n_estimators=10;, score=0.605 total time=   0.0s
[CV 3/5] END ......max_depth=3, n_estimators=10;, score=0.665 total time=   0.0s
[CV 4/5] END ......max_depth=3, n_estimators=10;, score=0.646 total time=   0.0s
[CV 5/5] END ......max_depth=3, n_estimators=10;, score=0.648 total time=   0.0s
[CV 1/5] END ......max_depth=3, n_estimators=50;, score=0.630 total time=   0.1s
[CV 2/5] END ......max_depth=3, n_estimators=50;, score=0.636 total time=   0.1s
[CV 3/5] END ......max_depth=3, n_estimators=50;, score=0.634 total time=   0.1s
[CV 4/5] END ......max_depth=3, n_estimators=50;, score=0.648 total time=   0.1s
[CV 5/5] END ......max_depth=3, n_estimators=50;, score=0.629 total time=   0.1s
[CV 1/5] END .....max_depth=3, n_estimators=100;, score=0.631 total time=   0.3s
[CV 2/5] END .....max_depth=3, n_estimators=100;

INFO:root:[Random Forest] 🔍 Best Accuracy: 0.6600
[Random Forest] 🔍 Best Params: {'max_depth': 5, 'n_estimators': 1000}


### 🔍 Random Forest Metrics

In [19]:
metrics = evaluate_metrics(model, ufc_data)

### 📝 Log Training Results

In [20]:
log_training_result(model_name, model.best_params_, metrics, duration)

INFO:src.helpers:✅ Training logged to ../data/results/training_log.csv


### 💾 Save Model 

In [21]:
save_model(model, name=file_model_name[model_name])

INFO:src.io_model:✅ Model 'Random Forest' saved to: /home/mlioi/ufc-predictor/models/rf_best.pkl


##  AdaBoost Model

### 🚀 AdaBoost Training

In [22]:
model_name = 'AdaBoost'
start = time.time()
model = model_factory(model_name, ufc_data, model_params)
end = time.time()
duration = end - start

INFO:src.model_factory:[AdaBoost] UFC GridSearchCV Training (Accuracy)
INFO:root:[AdaBoost] 🤖 Training...


Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5] END learning_rate=0.01, n_estimators=10;, score=0.650 total time=   0.1s
[CV 2/5] END learning_rate=0.01, n_estimators=10;, score=0.640 total time=   0.1s
[CV 3/5] END learning_rate=0.01, n_estimators=10;, score=0.648 total time=   0.1s
[CV 4/5] END learning_rate=0.01, n_estimators=10;, score=0.647 total time=   0.1s
[CV 5/5] END learning_rate=0.01, n_estimators=10;, score=0.610 total time=   0.1s
[CV 1/5] END learning_rate=0.01, n_estimators=50;, score=0.650 total time=   0.3s
[CV 2/5] END learning_rate=0.01, n_estimators=50;, score=0.640 total time=   0.3s
[CV 3/5] END learning_rate=0.01, n_estimators=50;, score=0.648 total time=   0.3s
[CV 4/5] END learning_rate=0.01, n_estimators=50;, score=0.647 total time=   0.3s
[CV 5/5] END learning_rate=0.01, n_estimators=50;, score=0.610 total time=   0.3s
[CV 1/5] END learning_rate=0.01, n_estimators=100;, score=0.650 total time=   0.6s
[CV 2/5] END learning_rate=0.01, n_es

INFO:root:[AdaBoost] 🔍 Best Accuracy: 0.6663
[AdaBoost] 🔍 Best Params: {'learning_rate': 0.1, 'n_estimators': 100}


### 🔍 AdaBoost Metrics

In [23]:
metrics = evaluate_metrics(model, ufc_data)

### 📝 Log Training Results

In [24]:
log_training_result(model_name, model.best_params_, metrics, duration)

INFO:src.helpers:✅ Training logged to ../data/results/training_log.csv


### 💾 Save Model 

In [25]:
save_model(model, name=file_model_name[model_name])

INFO:src.io_model:✅ Model 'AdaBoost' saved to: /home/mlioi/ufc-predictor/models/ab_best.pkl


## Naive Bayes Model

### 🚀 Naive Bayes Training

In [26]:
model_name = 'Naive Bayes'
start = time.time()
model = model_factory(model_name, ufc_data, model_params)
end = time.time()
duration = end - start

INFO:src.model_factory:[Naive Bayes] UFC GridSearchCV Training (Accuracy)
INFO:root:[Naive Bayes] 🤖 Training...


Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 1/5] END ...............var_smoothing=1e-08;, score=0.643 total time=   0.0s
[CV 2/5] END ...............var_smoothing=1e-08;, score=0.629 total time=   0.0s
[CV 3/5] END ...............var_smoothing=1e-08;, score=0.630 total time=   0.0s
[CV 4/5] END ...............var_smoothing=1e-08;, score=0.621 total time=   0.0s
[CV 5/5] END ...............var_smoothing=1e-08;, score=0.642 total time=   0.0s
[CV 1/5] END ...............var_smoothing=1e-07;, score=0.643 total time=   0.0s
[CV 2/5] END ...............var_smoothing=1e-07;, score=0.629 total time=   0.0s
[CV 3/5] END ...............var_smoothing=1e-07;, score=0.630 total time=   0.0s
[CV 4/5] END ...............var_smoothing=1e-07;, score=0.621 total time=   0.0s
[CV 5/5] END ...............var_smoothing=1e-07;, score=0.642 total time=   0.0s
[CV 1/5] END ...............var_smoothing=1e-06;, score=0.643 total time=   0.0s
[CV 2/5] END ...............var_smoothing=1e-06;,

INFO:root:[Naive Bayes] 🔍 Best Accuracy: 0.6329
[Naive Bayes] 🔍 Best Params: {'var_smoothing': 1e-08}


### 🔍 Naive Bayes Metrics

In [27]:
metrics = evaluate_metrics(model, ufc_data)

### 📝 Log Training Results

In [28]:
log_training_result(model_name, model.best_params_, metrics, duration)

INFO:src.helpers:✅ Training logged to ../data/results/training_log.csv


### 💾 Save Model 

In [29]:
save_model(model, name=file_model_name[model_name])

INFO:src.io_model:✅ Model 'Naive Bayes' saved to: /home/mlioi/ufc-predictor/models/nb_best.pkl


## Gradient Boosting Model

### 🚀 Gradient Boosting Training

In [30]:
model_name = "Gradient Boosting"
start = time.time()
model = model_factory(model_name, ufc_data, model_params)
end = time.time()
duration = end - start

INFO:src.model_factory:[Gradient Boosting] UFC GridSearchCV Training (Accuracy)
INFO:root:[Gradient Boosting] 🤖 Training...


Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV 1/5] END learning_rate=0.01, max_depth=3, n_estimators=50;, score=0.637 total time=   0.6s
[CV 2/5] END learning_rate=0.01, max_depth=3, n_estimators=50;, score=0.642 total time=   0.6s
[CV 3/5] END learning_rate=0.01, max_depth=3, n_estimators=50;, score=0.681 total time=   0.6s
[CV 4/5] END learning_rate=0.01, max_depth=3, n_estimators=50;, score=0.650 total time=   0.6s
[CV 5/5] END learning_rate=0.01, max_depth=3, n_estimators=50;, score=0.644 total time=   0.6s
[CV 1/5] END learning_rate=0.01, max_depth=3, n_estimators=100;, score=0.642 total time=   1.1s
[CV 2/5] END learning_rate=0.01, max_depth=3, n_estimators=100;, score=0.639 total time=   1.1s
[CV 3/5] END learning_rate=0.01, max_depth=3, n_estimators=100;, score=0.699 total time=   1.1s
[CV 4/5] END learning_rate=0.01, max_depth=3, n_estimators=100;, score=0.666 total time=   1.1s
[CV 5/5] END learning_rate=0.01, max_depth=3, n_estimators=100;, score=0.652 tot

INFO:root:[Gradient Boosting] 🔍 Best Accuracy: 0.6594
[Gradient Boosting] 🔍 Best Params: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100}


### 🔍 Gradient Boosting Metrics

In [31]:
metrics = evaluate_metrics(model, ufc_data)

### 📝 Log Training Results

In [32]:
log_training_result(model_name, model.best_params_, metrics, duration)

INFO:src.helpers:✅ Training logged to ../data/results/training_log.csv


### 💾 Save Model 

In [33]:
save_model(model, name=file_model_name[model_name])

INFO:src.io_model:✅ Model 'Gradient Boosting' saved to: /home/mlioi/ufc-predictor/models/gb_best.pkl


## Extra Trees Model

### 🚀 Extra Trees Training

In [34]:
model_name = "Extra Trees"
start = time.time()
model = model_factory(model_name, ufc_data, model_params)
end = time.time()
duration = end - start

INFO:src.model_factory:[Extra Trees] UFC GridSearchCV Training (Accuracy)
INFO:root:[Extra Trees] 🤖 Training...


Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5] END ...max_depth=None, n_estimators=50;, score=0.631 total time=   0.3s
[CV 2/5] END ...max_depth=None, n_estimators=50;, score=0.633 total time=   0.3s
[CV 3/5] END ...max_depth=None, n_estimators=50;, score=0.655 total time=   0.3s
[CV 4/5] END ...max_depth=None, n_estimators=50;, score=0.645 total time=   0.3s
[CV 5/5] END ...max_depth=None, n_estimators=50;, score=0.610 total time=   0.3s
[CV 1/5] END ..max_depth=None, n_estimators=100;, score=0.639 total time=   0.5s
[CV 2/5] END ..max_depth=None, n_estimators=100;, score=0.626 total time=   0.5s
[CV 3/5] END ..max_depth=None, n_estimators=100;, score=0.674 total time=   0.5s
[CV 4/5] END ..max_depth=None, n_estimators=100;, score=0.641 total time=   0.6s
[CV 5/5] END ..max_depth=None, n_estimators=100;, score=0.624 total time=   0.5s
[CV 1/5] END .....max_depth=10, n_estimators=50;, score=0.624 total time=   0.1s
[CV 2/5] END .....max_depth=10, n_estimators=50;,

INFO:root:[Extra Trees] 🔍 Best Accuracy: 0.6444
[Extra Trees] 🔍 Best Params: {'max_depth': 20, 'n_estimators': 100}


### 🔍 Extra Trees Metrics

In [35]:
metrics = evaluate_metrics(model, ufc_data)

### 📝 Log Training Results

In [36]:
log_training_result(model_name, model.best_params_, metrics, duration)

INFO:src.helpers:✅ Training logged to ../data/results/training_log.csv


### 💾 Save Model 

In [37]:
save_model(model, name=file_model_name[model_name])

INFO:src.io_model:✅ Model 'Extra Trees' saved to: /home/mlioi/ufc-predictor/models/et_best.pkl


## Quadratic Discriminant Analysis Model

### 🚀 Quadratic Discriminant Analysis Training

In [38]:
model_name = "Quadratic Discriminant Analysis"
start = time.time()
model = model_factory(model_name, ufc_data, model_params)
end = time.time()
duration = end - start

INFO:src.model_factory:[Quadratic Discriminant Analysis] UFC GridSearchCV Training (Accuracy)
INFO:root:[Quadratic Discriminant Analysis] 🤖 Training...


Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV 1/5] END .....................reg_param=0.1;, score=0.632 total time=   0.0s
[CV 2/5] END .....................reg_param=0.1;, score=0.637 total time=   0.0s
[CV 3/5] END .....................reg_param=0.1;, score=0.665 total time=   0.0s
[CV 4/5] END .....................reg_param=0.1;, score=0.624 total time=   0.0s
[CV 5/5] END .....................reg_param=0.1;, score=0.641 total time=   0.0s
[CV 1/5] END .......................reg_param=1;, score=0.654 total time=   0.0s
[CV 2/5] END .......................reg_param=1;, score=0.626 total time=   0.0s
[CV 3/5] END .......................reg_param=1;, score=0.648 total time=   0.0s
[CV 4/5] END .......................reg_param=1;, score=0.642 total time=   0.0s
[CV 5/5] END .......................reg_param=1;, score=0.658 total time=   0.0s


INFO:root:[Quadratic Discriminant Analysis] 🔍 Best Accuracy: 0.6456
[Quadratic Discriminant Analysis] 🔍 Best Params: {'reg_param': 1}


### 🔍 Quadratic Discriminant Analysis Metrics

In [39]:
metrics = evaluate_metrics(model, ufc_data)

### 📝 Log Training Results

In [40]:
log_training_result(model_name, model.best_params_, metrics, duration)

INFO:src.helpers:✅ Training logged to ../data/results/training_log.csv


### 💾 Save Model 

In [41]:
save_model(model, name=file_model_name[model_name])

INFO:src.io_model:✅ Model 'Quadratic Discriminant Analysis' saved to: /home/mlioi/ufc-predictor/models/qda_best.pkl


## Neural Network

### 🚀 Neural Network Training 

In [42]:
model_name = 'Neural Network'
start = time.time()
model = model_factory(model_name, ufc_data, model_params)
end = time.time()
duration = end - start

INFO:src.model_factory:[Neural Network] UFC GridSearchCV Training (Accuracy)
INFO:root:[Neural Network] 🤖 Training...


Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV 1/5] END activation=relu, alpha=0.001, batch_size=32, early_stopping=True, hidden_layer_sizes=(200,), learning_rate=adaptive, learning_rate_init=0.01, momentum=0.8, solver=adam, validation_fraction=0.2;, score=0.654 total time=   0.8s
[CV 2/5] END activation=relu, alpha=0.001, batch_size=32, early_stopping=True, hidden_layer_sizes=(200,), learning_rate=adaptive, learning_rate_init=0.01, momentum=0.8, solver=adam, validation_fraction=0.2;, score=0.644 total time=   0.6s
[CV 3/5] END activation=relu, alpha=0.001, batch_size=32, early_stopping=True, hidden_layer_sizes=(200,), learning_rate=adaptive, learning_rate_init=0.01, momentum=0.8, solver=adam, validation_fraction=0.2;, score=0.662 total time=   0.6s
[CV 4/5] END activation=relu, alpha=0.001, batch_size=32, early_stopping=True, hidden_layer_sizes=(200,), learning_rate=adaptive, learning_rate_init=0.01, momentum=0.8, solver=adam, validation_fraction=0.2;, score=0.645 t

INFO:root:[Neural Network] 🔍 Best Accuracy: 0.6621
[Neural Network] 🔍 Best Params: {'activation': 'logistic', 'alpha': 0.001, 'batch_size': 32, 'early_stopping': True, 'hidden_layer_sizes': (50, 50), 'learning_rate': 'adaptive', 'learning_rate_init': 0.01, 'momentum': 0.8, 'solver': 'adam', 'validation_fraction': 0.2}


### 🔍 Neural Network Metrics

In [43]:
metrics = evaluate_metrics(model, ufc_data)

### 📝 Log Training Results

In [44]:
log_training_result(model_name, model.best_params_, metrics, duration)

INFO:src.helpers:✅ Training logged to ../data/results/training_log.csv


### 💾 Save Model 

In [45]:
save_model(model, name=file_model_name[model_name])

INFO:src.io_model:✅ Model 'Neural Network' saved to: /home/mlioi/ufc-predictor/models/nn_best.pkl


## XGBoost Model

### 🚀 XGBoost Training 

In [46]:
model_name = 'XGBoost'
start = time.time()
model = model_factory(model_name, ufc_data, model_params)
end = time.time()
duration = end - start

INFO:src.model_factory:[XGBoost] UFC GridSearchCV Training (Accuracy)
INFO:root:[XGBoost] 🤖 Training...


Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV 1/5] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.8;, score=0.626 total time=   0.1s
[CV 2/5] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.8;, score=0.620 total time=   0.2s
[CV 3/5] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.8;, score=0.617 total time=   0.1s
[CV 4/5] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.8;, score=0.611 total time=   0.2s
[CV 5/5] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.8;, score=0.636 total time=   0.1s
[CV 1/5] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=1.0;, score=0.624 total time=   0.1s
[CV 2/5] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=1.0;, score=0.628 total time=   0.1s
[CV 3/5] END c

INFO:root:[XGBoost] 🔍 Best Accuracy: 0.6635
[XGBoost] 🔍 Best Params: {'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 50, 'subsample': 0.8}


### 🔍 XGBoost Metrics

In [47]:
metrics = evaluate_metrics(model, ufc_data)

### 📝 Log Training Results

In [48]:
log_training_result(model_name, model.best_params_, metrics, duration)

INFO:src.helpers:✅ Training logged to ../data/results/training_log.csv


### 💾 Save Model 

In [49]:
save_model(model, name=file_model_name[model_name])

INFO:src.io_model:✅ Model 'XGBoost' saved to: /home/mlioi/ufc-predictor/models/xgb_best.pkl


# 🔹 UFC Machine Learning Training (without Odds)

##  KNN Model Training

### 🚀 KNN Training 

In [50]:
model_name = 'K-Nearest Neighbors'
start = time.time()
model = model_factory(model_name, ufc_data_no_odds, model_params, scoring = 'accuracy')
end = time.time()
duration = end - start

INFO:src.model_factory:[K-Nearest Neighbors] UFC GridSearchCV Training (Accuracy)
INFO:root:[K-Nearest Neighbors] 🤖 Training...


Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV 1/5] END metric=euclidean, n_neighbors=3, weights=uniform;, score=0.546 total time=   0.1s
[CV 2/5] END metric=euclidean, n_neighbors=3, weights=uniform;, score=0.570 total time=   0.1s
[CV 3/5] END metric=euclidean, n_neighbors=3, weights=uniform;, score=0.566 total time=   0.1s
[CV 4/5] END metric=euclidean, n_neighbors=3, weights=uniform;, score=0.562 total time=   0.1s
[CV 5/5] END metric=euclidean, n_neighbors=3, weights=uniform;, score=0.553 total time=   0.1s
[CV 1/5] END metric=euclidean, n_neighbors=3, weights=distance;, score=0.546 total time=   0.0s
[CV 2/5] END metric=euclidean, n_neighbors=3, weights=distance;, score=0.574 total time=   0.0s
[CV 3/5] END metric=euclidean, n_neighbors=3, weights=distance;, score=0.567 total time=   0.0s
[CV 4/5] END metric=euclidean, n_neighbors=3, weights=distance;, score=0.560 total time=   0.0s
[CV 5/5] END metric=euclidean, n_neighbors=3, weights=distance;, score=0.552 to

INFO:root:[K-Nearest Neighbors] 🔍 Best Accuracy: 0.5721
[K-Nearest Neighbors] 🔍 Best Params: {'metric': 'euclidean', 'n_neighbors': 12, 'weights': 'distance'}


### 🔍 KNN Metrics

In [51]:
metrics = evaluate_metrics(model, ufc_data_no_odds)

### 📝 Log Training Results

In [53]:
log_training_result(f"{model_name} (without odds)", model.best_params_, metrics, duration)

INFO:src.helpers:✅ Training logged to ../data/results/training_log.csv


### 💾 Save Model 

In [54]:
save_model(model, name=f"{file_model_name[model_name]}_no_odds")

INFO:src.io_model:✅ Model 'K-Nearest Neighbors' saved to: /home/mlioi/ufc-predictor/models/knn_best_no_odds.pkl


## Support Vector Machine Model 

### 🚀 Support Vector Machine Training 

In [6]:
model_name = 'Support Vector Machine'
start = time.time()
model = model_factory(model_name, ufc_data_no_odds, model_params)
end = time.time()
duration = end - start

INFO:src.model_factory:[Support Vector Machine] UFC GridSearchCV Training (Accuracy)
INFO:root:[Support Vector Machine] 🤖 Training...


Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END .C=0.01, gamma=auto, kernel=linear;, score=0.581 total time=   2.8s
[CV 2/5] END .C=0.01, gamma=auto, kernel=linear;, score=0.581 total time=   2.8s
[CV 3/5] END .C=0.01, gamma=auto, kernel=linear;, score=0.580 total time=   3.1s
[CV 4/5] END .C=0.01, gamma=auto, kernel=linear;, score=0.580 total time=   2.9s
[CV 5/5] END .C=0.01, gamma=auto, kernel=linear;, score=0.580 total time=   2.9s
[CV 1/5] END ....C=0.01, gamma=auto, kernel=rbf;, score=0.581 total time=   4.2s
[CV 2/5] END ....C=0.01, gamma=auto, kernel=rbf;, score=0.581 total time=   4.1s
[CV 3/5] END ....C=0.01, gamma=auto, kernel=rbf;, score=0.580 total time=   4.2s
[CV 4/5] END ....C=0.01, gamma=auto, kernel=rbf;, score=0.580 total time=   4.3s
[CV 5/5] END ....C=0.01, gamma=auto, kernel=rbf;, score=0.580 total time=   4.2s
[CV 1/5] END ..C=0.1, gamma=auto, kernel=linear;, score=0.581 total time=   3.3s
[CV 2/5] END ..C=0.1, gamma=auto, kernel=linear;

INFO:root:[Support Vector Machine] 🔍 Best Accuracy: 0.5988
[Support Vector Machine] 🔍 Best Params: {'C': 1, 'gamma': 'auto', 'kernel': 'rbf'}


### 🔍 Support Vector Machine Metrics

In [7]:
metrics = evaluate_metrics(model, ufc_data_no_odds)

### 📝 Log Training Results

In [8]:
log_training_result(f"{model_name} (without odds)", model.best_params_, metrics, duration)

INFO:src.helpers:✅ Training logged to ../data/results/training_log.csv


### 💾 Save Model 

In [9]:
save_model(model, name=f"{file_model_name[model_name]}_no_odds")

INFO:src.io_model:✅ Model 'Support Vector Machine' saved to: /home/mlioi/ufc-predictor/models/svm_best_no_odds.pkl


## Logistic Regression Model

### 🚀 Logistic Regression Training

In [59]:
model_name = 'Logistic Regression'
start = time.time()
model = model_factory(model_name, ufc_data_no_odds, model_params)
end = time.time()
duration = end - start

INFO:src.model_factory:[Logistic Regression] UFC GridSearchCV Training (Accuracy)
INFO:root:[Logistic Regression] 🤖 Training...


Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV 1/5] END C=0.001, penalty=l1, solver=liblinear;, score=0.581 total time=   0.0s
[CV 2/5] END C=0.001, penalty=l1, solver=liblinear;, score=0.581 total time=   0.0s
[CV 3/5] END C=0.001, penalty=l1, solver=liblinear;, score=0.580 total time=   0.0s
[CV 4/5] END C=0.001, penalty=l1, solver=liblinear;, score=0.580 total time=   0.0s
[CV 5/5] END C=0.001, penalty=l1, solver=liblinear;, score=0.580 total time=   0.0s
[CV 1/5] END C=0.001, penalty=l2, solver=liblinear;, score=0.598 total time=   0.0s
[CV 2/5] END C=0.001, penalty=l2, solver=liblinear;, score=0.561 total time=   0.0s
[CV 3/5] END C=0.001, penalty=l2, solver=liblinear;, score=0.602 total time=   0.0s
[CV 4/5] END C=0.001, penalty=l2, solver=liblinear;, score=0.593 total time=   0.0s
[CV 5/5] END C=0.001, penalty=l2, solver=liblinear;, score=0.598 total time=   0.0s
[CV 1/5] END C=0.01, penalty=l1, solver=liblinear;, score=0.598 total time=   0.0s
[CV 2/5] END C=0.

INFO:root:[Logistic Regression] 🔍 Best Accuracy: 0.5977
[Logistic Regression] 🔍 Best Params: {'C': 0.05, 'penalty': 'l2', 'solver': 'liblinear'}


### 🔍 Logistic Regression Metrics

In [60]:
metrics = evaluate_metrics(model, ufc_data_no_odds)

### 📝 Log Training Results

In [61]:
log_training_result(f"{model_name} (without odds)", model.best_params_, metrics, duration)

INFO:src.helpers:✅ Training logged to ../data/results/training_log.csv


### 💾 Save Model 

In [62]:
save_model(model, name=f"{file_model_name[model_name]}_no_odds")

INFO:src.io_model:✅ Model 'Logistic Regression' saved to: /home/mlioi/ufc-predictor/models/lr_best_no_odds.pkl


## Random Forest Model

### 🚀 Random Forest Training

In [63]:
model_name = 'Random Forest'
start = time.time()
model = model_factory(model_name, ufc_data_no_odds, model_params)
end = time.time()
duration = end - start

INFO:src.model_factory:[Random Forest] UFC GridSearchCV Training (Accuracy)
INFO:root:[Random Forest] 🤖 Training...


Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV 1/5] END ......max_depth=3, n_estimators=10;, score=0.588 total time=   0.0s
[CV 2/5] END ......max_depth=3, n_estimators=10;, score=0.580 total time=   0.0s
[CV 3/5] END ......max_depth=3, n_estimators=10;, score=0.592 total time=   0.0s
[CV 4/5] END ......max_depth=3, n_estimators=10;, score=0.582 total time=   0.0s
[CV 5/5] END ......max_depth=3, n_estimators=10;, score=0.590 total time=   0.0s
[CV 1/5] END ......max_depth=3, n_estimators=50;, score=0.595 total time=   0.1s
[CV 2/5] END ......max_depth=3, n_estimators=50;, score=0.584 total time=   0.1s
[CV 3/5] END ......max_depth=3, n_estimators=50;, score=0.593 total time=   0.1s
[CV 4/5] END ......max_depth=3, n_estimators=50;, score=0.594 total time=   0.1s
[CV 5/5] END ......max_depth=3, n_estimators=50;, score=0.599 total time=   0.1s
[CV 1/5] END .....max_depth=3, n_estimators=100;, score=0.583 total time=   0.3s
[CV 2/5] END .....max_depth=3, n_estimators=100;

INFO:root:[Random Forest] 🔍 Best Accuracy: 0.6029
[Random Forest] 🔍 Best Params: {'max_depth': 10, 'n_estimators': 1000}


### 🔍 Random Forest Metrics

In [64]:
metrics = evaluate_metrics(model, ufc_data_no_odds)

### 📝 Log Training Results

In [65]:
log_training_result(f"{model_name} (without odds)", model.best_params_, metrics, duration)

INFO:src.helpers:✅ Training logged to ../data/results/training_log.csv


### 💾 Save Model 

In [66]:
save_model(model, name=f"{file_model_name[model_name]}_no_odds")

INFO:src.io_model:✅ Model 'Random Forest' saved to: /home/mlioi/ufc-predictor/models/rf_best_no_odds.pkl


##  AdaBoost Model

### 🚀 AdaBoost Training

In [67]:
model_name = 'AdaBoost'
start = time.time()
model = model_factory(model_name, ufc_data_no_odds, model_params)
end = time.time()
duration = end - start

INFO:src.model_factory:[AdaBoost] UFC GridSearchCV Training (Accuracy)
INFO:root:[AdaBoost] 🤖 Training...


Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5] END learning_rate=0.01, n_estimators=10;, score=0.581 total time=   0.1s
[CV 2/5] END learning_rate=0.01, n_estimators=10;, score=0.581 total time=   0.1s
[CV 3/5] END learning_rate=0.01, n_estimators=10;, score=0.580 total time=   0.1s
[CV 4/5] END learning_rate=0.01, n_estimators=10;, score=0.580 total time=   0.1s
[CV 5/5] END learning_rate=0.01, n_estimators=10;, score=0.580 total time=   0.1s
[CV 1/5] END learning_rate=0.01, n_estimators=50;, score=0.581 total time=   0.3s
[CV 2/5] END learning_rate=0.01, n_estimators=50;, score=0.581 total time=   0.3s
[CV 3/5] END learning_rate=0.01, n_estimators=50;, score=0.580 total time=   0.3s
[CV 4/5] END learning_rate=0.01, n_estimators=50;, score=0.580 total time=   0.3s
[CV 5/5] END learning_rate=0.01, n_estimators=50;, score=0.580 total time=   0.3s
[CV 1/5] END learning_rate=0.01, n_estimators=100;, score=0.581 total time=   0.5s
[CV 2/5] END learning_rate=0.01, n_es

INFO:root:[AdaBoost] 🔍 Best Accuracy: 0.5948
[AdaBoost] 🔍 Best Params: {'learning_rate': 1.0, 'n_estimators': 50}


### 🔍 AdaBoost Metrics

In [68]:
metrics = evaluate_metrics(model, ufc_data_no_odds)

### 📝 Log Training Results

In [69]:
log_training_result(f"{model_name} (without odds)", model.best_params_, metrics, duration)

INFO:src.helpers:✅ Training logged to ../data/results/training_log.csv


### 💾 Save Model 

In [70]:
save_model(model, name=f"{file_model_name[model_name]}_no_odds")

INFO:src.io_model:✅ Model 'AdaBoost' saved to: /home/mlioi/ufc-predictor/models/ab_best_no_odds.pkl


## Naive Bayes Model

### 🚀 Naive Bayes Training

In [71]:
model_name = 'Naive Bayes'
start = time.time()
model = model_factory(model_name, ufc_data_no_odds, model_params)
end = time.time()
duration = end - start

INFO:src.model_factory:[Naive Bayes] UFC GridSearchCV Training (Accuracy)
INFO:root:[Naive Bayes] 🤖 Training...


Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 1/5] END ...............var_smoothing=1e-08;, score=0.565 total time=   0.0s
[CV 2/5] END ...............var_smoothing=1e-08;, score=0.548 total time=   0.0s
[CV 3/5] END ...............var_smoothing=1e-08;, score=0.551 total time=   0.0s
[CV 4/5] END ...............var_smoothing=1e-08;, score=0.542 total time=   0.0s
[CV 5/5] END ...............var_smoothing=1e-08;, score=0.548 total time=   0.0s
[CV 1/5] END ...............var_smoothing=1e-07;, score=0.565 total time=   0.0s
[CV 2/5] END ...............var_smoothing=1e-07;, score=0.548 total time=   0.0s
[CV 3/5] END ...............var_smoothing=1e-07;, score=0.551 total time=   0.0s
[CV 4/5] END ...............var_smoothing=1e-07;, score=0.542 total time=   0.0s
[CV 5/5] END ...............var_smoothing=1e-07;, score=0.548 total time=   0.0s
[CV 1/5] END ...............var_smoothing=1e-06;, score=0.565 total time=   0.0s
[CV 2/5] END ...............var_smoothing=1e-06;,

INFO:root:[Naive Bayes] 🔍 Best Accuracy: 0.5506
[Naive Bayes] 🔍 Best Params: {'var_smoothing': 1e-08}


### 🔍 Naive Bayes Metrics

In [72]:
metrics = evaluate_metrics(model, ufc_data_no_odds)

### 📝 Log Training Results

In [73]:
log_training_result(f"{model_name} (without odds)", model.best_params_, metrics, duration)

INFO:src.helpers:✅ Training logged to ../data/results/training_log.csv


### 💾 Save Model 

In [74]:
save_model(model, name=f"{file_model_name[model_name]}_no_odds")

INFO:src.io_model:✅ Model 'Naive Bayes' saved to: /home/mlioi/ufc-predictor/models/nb_best_no_odds.pkl


## Gradient Boosting Model

### 🚀 Gradient Boosting Training

In [75]:
model_name = "Gradient Boosting"
start = time.time()
model = model_factory(model_name, ufc_data_no_odds, model_params)
end = time.time()
duration = end - start

INFO:src.model_factory:[Gradient Boosting] UFC GridSearchCV Training (Accuracy)
INFO:root:[Gradient Boosting] 🤖 Training...


Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV 1/5] END learning_rate=0.01, max_depth=3, n_estimators=50;, score=0.584 total time=   0.5s
[CV 2/5] END learning_rate=0.01, max_depth=3, n_estimators=50;, score=0.585 total time=   0.5s
[CV 3/5] END learning_rate=0.01, max_depth=3, n_estimators=50;, score=0.583 total time=   0.5s
[CV 4/5] END learning_rate=0.01, max_depth=3, n_estimators=50;, score=0.583 total time=   0.5s
[CV 5/5] END learning_rate=0.01, max_depth=3, n_estimators=50;, score=0.585 total time=   0.5s
[CV 1/5] END learning_rate=0.01, max_depth=3, n_estimators=100;, score=0.591 total time=   1.0s
[CV 2/5] END learning_rate=0.01, max_depth=3, n_estimators=100;, score=0.584 total time=   1.0s
[CV 3/5] END learning_rate=0.01, max_depth=3, n_estimators=100;, score=0.595 total time=   1.0s
[CV 4/5] END learning_rate=0.01, max_depth=3, n_estimators=100;, score=0.592 total time=   1.0s
[CV 5/5] END learning_rate=0.01, max_depth=3, n_estimators=100;, score=0.590 tot

INFO:root:[Gradient Boosting] 🔍 Best Accuracy: 0.5990
[Gradient Boosting] 🔍 Best Params: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 50}


### 🔍 Gradient Boosting Metrics

In [76]:
metrics = evaluate_metrics(model, ufc_data_no_odds)

### 📝 Log Training Results

In [77]:
log_training_result(f"{model_name} (without odds)", model.best_params_, metrics, duration)

INFO:src.helpers:✅ Training logged to ../data/results/training_log.csv


### 💾 Save Model 

In [78]:
save_model(model, name=f"{file_model_name[model_name]}_no_odds")

INFO:src.io_model:✅ Model 'Gradient Boosting' saved to: /home/mlioi/ufc-predictor/models/gb_best_no_odds.pkl


## Extra Trees Model

### 🚀 Extra Trees Training

In [79]:
model_name = "Extra Trees"
start = time.time()
model = model_factory(model_name, ufc_data_no_odds, model_params)
end = time.time()
duration = end - start

INFO:src.model_factory:[Extra Trees] UFC GridSearchCV Training (Accuracy)
INFO:root:[Extra Trees] 🤖 Training...


Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5] END ...max_depth=None, n_estimators=50;, score=0.568 total time=   0.3s
[CV 2/5] END ...max_depth=None, n_estimators=50;, score=0.567 total time=   0.3s
[CV 3/5] END ...max_depth=None, n_estimators=50;, score=0.583 total time=   0.3s
[CV 4/5] END ...max_depth=None, n_estimators=50;, score=0.578 total time=   0.3s
[CV 5/5] END ...max_depth=None, n_estimators=50;, score=0.565 total time=   0.3s
[CV 1/5] END ..max_depth=None, n_estimators=100;, score=0.598 total time=   0.5s
[CV 2/5] END ..max_depth=None, n_estimators=100;, score=0.577 total time=   0.6s
[CV 3/5] END ..max_depth=None, n_estimators=100;, score=0.601 total time=   0.6s
[CV 4/5] END ..max_depth=None, n_estimators=100;, score=0.582 total time=   0.6s
[CV 5/5] END ..max_depth=None, n_estimators=100;, score=0.579 total time=   0.6s
[CV 1/5] END .....max_depth=10, n_estimators=50;, score=0.594 total time=   0.1s
[CV 2/5] END .....max_depth=10, n_estimators=50;,

INFO:root:[Extra Trees] 🔍 Best Accuracy: 0.5915
[Extra Trees] 🔍 Best Params: {'max_depth': 10, 'n_estimators': 50}


### 🔍 Extra Trees Metrics

In [80]:
metrics = evaluate_metrics(model, ufc_data_no_odds)

### 📝 Log Training Results

In [81]:
log_training_result(f"{model_name} (without odds)", model.best_params_, metrics, duration)

INFO:src.helpers:✅ Training logged to ../data/results/training_log.csv


### 💾 Save Model 

In [82]:
save_model(model, name=f"{file_model_name[model_name]}_no_odds")

INFO:src.io_model:✅ Model 'Extra Trees' saved to: /home/mlioi/ufc-predictor/models/et_best_no_odds.pkl


## Quadratic Discriminant Analysis Model

### 🚀 Quadratic Discriminant Analysis Training

In [83]:
model_name = "Quadratic Discriminant Analysis"
start = time.time()
model = model_factory(model_name, ufc_data_no_odds, model_params)
end = time.time()
duration = end - start

INFO:src.model_factory:[Quadratic Discriminant Analysis] UFC GridSearchCV Training (Accuracy)
INFO:root:[Quadratic Discriminant Analysis] 🤖 Training...


Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV 1/5] END .....................reg_param=0.1;, score=0.564 total time=   0.0s
[CV 2/5] END .....................reg_param=0.1;, score=0.543 total time=   0.0s
[CV 3/5] END .....................reg_param=0.1;, score=0.578 total time=   0.0s
[CV 4/5] END .....................reg_param=0.1;, score=0.560 total time=   0.0s
[CV 5/5] END .....................reg_param=0.1;, score=0.548 total time=   0.0s
[CV 1/5] END .......................reg_param=1;, score=0.601 total time=   0.0s
[CV 2/5] END .......................reg_param=1;, score=0.556 total time=   0.0s
[CV 3/5] END .......................reg_param=1;, score=0.596 total time=   0.0s
[CV 4/5] END .......................reg_param=1;, score=0.592 total time=   0.0s
[CV 5/5] END .......................reg_param=1;, score=0.619 total time=   0.0s


INFO:root:[Quadratic Discriminant Analysis] 🔍 Best Accuracy: 0.5927
[Quadratic Discriminant Analysis] 🔍 Best Params: {'reg_param': 1}


### 🔍 Quadratic Discriminant Analysis Metrics

In [84]:
metrics = evaluate_metrics(model, ufc_data_no_odds)

### 📝 Log Training Results

In [85]:
log_training_result(f"{model_name} (without odds)", model.best_params_, metrics, duration)

INFO:src.helpers:✅ Training logged to ../data/results/training_log.csv


### 💾 Save Model 

In [86]:
save_model(model, name=f"{file_model_name[model_name]}_no_odds")

INFO:src.io_model:✅ Model 'Quadratic Discriminant Analysis' saved to: /home/mlioi/ufc-predictor/models/qda_best_no_odds.pkl


## Neural Network

### 🚀 Neural Network Training 

In [87]:
model_name = 'Neural Network'
start = time.time()
model = model_factory(model_name, ufc_data_no_odds, model_params)
end = time.time()
duration = end - start

INFO:src.model_factory:[Neural Network] UFC GridSearchCV Training (Accuracy)
INFO:root:[Neural Network] 🤖 Training...


Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV 1/5] END activation=relu, alpha=0.001, batch_size=32, early_stopping=True, hidden_layer_sizes=(200,), learning_rate=adaptive, learning_rate_init=0.01, momentum=0.8, solver=adam, validation_fraction=0.2;, score=0.589 total time=   0.7s
[CV 2/5] END activation=relu, alpha=0.001, batch_size=32, early_stopping=True, hidden_layer_sizes=(200,), learning_rate=adaptive, learning_rate_init=0.01, momentum=0.8, solver=adam, validation_fraction=0.2;, score=0.558 total time=   1.1s
[CV 3/5] END activation=relu, alpha=0.001, batch_size=32, early_stopping=True, hidden_layer_sizes=(200,), learning_rate=adaptive, learning_rate_init=0.01, momentum=0.8, solver=adam, validation_fraction=0.2;, score=0.578 total time=   0.6s
[CV 4/5] END activation=relu, alpha=0.001, batch_size=32, early_stopping=True, hidden_layer_sizes=(200,), learning_rate=adaptive, learning_rate_init=0.01, momentum=0.8, solver=adam, validation_fraction=0.2;, score=0.586 t

INFO:root:[Neural Network] 🔍 Best Accuracy: 0.5992
[Neural Network] 🔍 Best Params: {'activation': 'relu', 'alpha': 0.01, 'batch_size': 64, 'early_stopping': True, 'hidden_layer_sizes': (200,), 'learning_rate': 'adaptive', 'learning_rate_init': 0.01, 'momentum': 0.8, 'solver': 'sgd', 'validation_fraction': 0.2}


### 🔍 Neural Network Metrics

In [88]:
metrics = evaluate_metrics(model, ufc_data_no_odds)

### 📝 Log Training Results

In [89]:
log_training_result(f"{model_name} (without odds)", model.best_params_, metrics, duration)

INFO:src.helpers:✅ Training logged to ../data/results/training_log.csv


### 💾 Save Model 

In [90]:
save_model(model, name=f"{file_model_name[model_name]}_no_odds")

INFO:src.io_model:✅ Model 'Neural Network' saved to: /home/mlioi/ufc-predictor/models/nn_best_no_odds.pkl


## XGBoost Model

### 🚀 XGBoost Training 

In [91]:
model_name = 'XGBoost'
start = time.time()
model = model_factory(model_name, ufc_data_no_odds, model_params)
end = time.time()
duration = end - start

INFO:src.model_factory:[XGBoost] UFC GridSearchCV Training (Accuracy)
INFO:root:[XGBoost] 🤖 Training...


Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV 1/5] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.8;, score=0.581 total time=   0.2s
[CV 2/5] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.8;, score=0.582 total time=   0.2s
[CV 3/5] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.8;, score=0.580 total time=   0.2s
[CV 4/5] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.8;, score=0.581 total time=   0.2s
[CV 5/5] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.8;, score=0.581 total time=   0.1s
[CV 1/5] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=1.0;, score=0.581 total time=   0.1s
[CV 2/5] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=1.0;, score=0.581 total time=   0.1s
[CV 3/5] END c

INFO:root:[XGBoost] 🔍 Best Accuracy: 0.6010
[XGBoost] 🔍 Best Params: {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 50, 'subsample': 0.8}


### 🔍 XGBoost Metrics

In [92]:
metrics = evaluate_metrics(model, ufc_data_no_odds)

### 📝 Log Training Results

In [93]:
log_training_result(f"{model_name} (without odds)", model.best_params_, metrics, duration)

INFO:src.helpers:✅ Training logged to ../data/results/training_log.csv


### 💾 Save Model 

In [94]:
save_model(model, name=f"{file_model_name[model_name]}_no_odds")

INFO:src.io_model:✅ Model 'XGBoost' saved to: /home/mlioi/ufc-predictor/models/xgb_best_no_odds.pkl


<div style="text-align: center;">
     <img src="../img/ufc_logo.png" width="800" /> 
</div>